In [ ]:
import torch
dtype = torch.float64
device = torch.device("cpu")
# torch.set_grad_enabled(False) # seems not to improve performance (maybe only reduces memory)
size = 1000000

In [ ]:
tc = torch.randn(size, device=device, dtype=dtype)
rh = torch.randn(size, device=device, dtype=dtype)

In [ ]:
# a 'simple' numpy to numpy function
def calculate_wbt_torch(tc, rh):
    tw = (
        tc * torch.arctan(0.151977 * torch.sqrt(rh + 8.313659))
        + torch.arctan(tc + rh)
        - torch.arctan(rh - 1.676331)
        + 0.00391838 * (rh) ** (3 / 2) * torch.arctan(0.023101 * rh)
        - 4.686035
    )
    return tw

In [ ]:
%timeit calculate_wbt_torch(tc, rh)

In [ ]:
# try Jax with more complex functions including (pure)functions that call (pure)functions 
import math

def kelvin_to_celsius(tk):
    tc = tk - 273.15
    return tc

def calculate_bgt_torch(t_k, mrt, va):
    f = (1.1e8 * va**0.6) / (0.98 * 0.15**0.4)
    a = f / 2
    b = -f * t_k - mrt**4
    rt1 = 3 ** (1 / 3)
    rt2 = math.sqrt(3) * torch.sqrt(27 * a**4 - 16 * b**3) + 9 * a**2
    rt3 = 2 * 2 ** (2 / 3) * b
    a = a.clip(min=0)
    bgt_quartic = -1 / 2 * torch.sqrt(
        rt3 / (rt1 * rt2 ** (1 / 3)) + (2 ** (1 / 3) * rt2 ** (1 / 3)) / 3 ** (2 / 3)
    ) + 1 / 2 * torch.sqrt(
        (4 * a)
        / torch.sqrt(
            rt3 / (rt1 * rt2 ** (1 / 3))
            + (2 ** (1 / 3) * rt2 ** (1 / 3)) / 3 ** (2 / 3)
        )
        - (2 ** (1 / 3) * rt2 ** (1 / 3)) / 3 ** (2 / 3)
        - rt3 / (rt1 * rt2 ** (1 / 3))
    )
    bgt_c = kelvin_to_celsius(bgt_quartic)
    return bgt_c


In [ ]:
tk = torch.randn(size, device=device, dtype=dtype)
mrt = torch.randn(size, device=device, dtype=dtype)
va = torch.randn(size, device=device, dtype=dtype)

In [ ]:
# torch doesn't do as well as say JAX because it was not able to compile and fuse the operations
%timeit calculate_bgt_torch(tk, mrt, va)